### Odds of getting your first choice

Here we compare Hartford and suburban application versus offers to see what the "acceptance rate" is for each group.

Applications and offers are separate spreadsheets, so this notebook aims to combine them on school name

In [116]:
import pandas as pd
from ct_schools import schools

### Import both spreadsheets

In [117]:
offers = pd.read_excel("data/CLEANED - Offers By School By Grade 2017-18.xlsx",
                       skiprows=1,
                       usecols=[0,1,2,3,4,5])
#offers = offers.set_index(["Operator","School","Grade"])
offers.head()

,Operator,School,Grade,N,N Hartford,N Suburban
0,BPS,Global Experience Magnet,6,72,20,52
1,BPS,Global Experience Magnet,7,9,NaN,9
2,BPS,Global Experience Magnet,8,11,NaN,11
3,BPS,Global Experience Magnet,9,25,11,14
4,BPS,Global Experience Magnet,10,11,NaN,11


In [118]:
# Clean up the column names for offers
offers.columns = ["operator","school","grade","n_offers","n_offers_hartford","n_offers_suburban"]
offers.head()

,operator,school,grade,n_offers,n_offers_hartford,n_offers_suburban
0,BPS,Global Experience Magnet,6,72,20,52
1,BPS,Global Experience Magnet,7,9,NaN,9
2,BPS,Global Experience Magnet,8,11,NaN,11
3,BPS,Global Experience Magnet,9,25,11,14
4,BPS,Global Experience Magnet,10,11,NaN,11


In [119]:
apps = pd.read_excel("data/CLEANED - First Chocie Apps By School By Grade 2017-18.xlsx",
                    skiprows=1)#.set_index(["Operator","School","Grade"])
apps.head()

,Operator,1st Choice School,Grade,N,N Hartford,N Suburban
0,BPS,Global Experience Magnet,6,32.0,7,25.0
1,BPS,Global Experience Magnet,7,13.0,NaN,NaN
2,BPS,Global Experience Magnet,8,6.0,NaN,NaN
3,BPS,Global Experience Magnet,9,19.0,10,9.0
4,BPS,Global Experience Magnet,10,9.0,NaN,NaN


In [120]:
apps.columns = ["operator","school","grade","n_apps","n_apps_hartford","n_apps_suburban"]
apps.head()

,operator,school,grade,n_apps,n_apps_hartford,n_apps_suburban
0,BPS,Global Experience Magnet,6,32.0,7,25.0
1,BPS,Global Experience Magnet,7,13.0,NaN,NaN
2,BPS,Global Experience Magnet,8,6.0,NaN,NaN
3,BPS,Global Experience Magnet,9,19.0,10,9.0
4,BPS,Global Experience Magnet,10,9.0,NaN,NaN


### Export data for use in open refine

In [126]:
apps["file"] = "apps"
offers["file"] = "offers"
pd.concat([apps,offers]).to_csv("data/FOR_REFINE_2.CSV",index=False)

### Merge the data

In [87]:
def combine(apps_df,offers_df):
    return apps_df.join(offers_df,how="outer",rsuffix="_offers")

combined = combine(apps,offers)
combined.head()

,operator,school,grade,n_apps,n_apps_hartford,n_apps_suburban,operator_offers,school_offers,grade_offers,n_offers,n_offers_hartford,n_offers_suburban
0,BPS,Global Experience Magnet,6,32.0,7,25.0,BPS,Global Experience Magnet,6,72,20,52
1,BPS,Global Experience Magnet,7,13.0,NaN,NaN,BPS,Global Experience Magnet,7,9,NaN,9
2,BPS,Global Experience Magnet,8,6.0,NaN,NaN,BPS,Global Experience Magnet,8,11,NaN,11
3,BPS,Global Experience Magnet,9,19.0,10,9.0,BPS,Global Experience Magnet,9,25,11,14
4,BPS,Global Experience Magnet,10,9.0,NaN,NaN,BPS,Global Experience Magnet,10,11,NaN,11


### Check the merge

Some rows in one spreadsheet didn't match rows in the other spreadsheet because the operator, school or grade didn't match up. Let's see where the problems were.

In [88]:
# Number of rows with apps but no offers data
no_offers = combined[combined["school_offers"].isnull()]
len(no_offers)

23

In [115]:
print len(offers)
print len(apps)

263
286


In [105]:
# There are definitely some OCR issues above. Let's use ct_schools.schools.closest() to try and find the closest match
def add_closest(df):
    ret = df.copy()
    ret["school_guess"] = ret["school"].apply(lambda x: schools.closest(x).iloc[0]["School Name"])
    ret["FUZZ"] = ret["school"].apply(lambda x: schools.closest(x).iloc[0]["FUZZ_RATIO"])

    return ret

apps = add_closest(apps)
offers = add_closest(offers)

In [89]:
# Here are the rows...
no_offers

,operator,school,grade,n_apps,n_apps_hartford,n_apps_suburban,operator_offers,school_offers,grade_offers,n_offers,n_offers_hartford,n_offers_suburban
263,HPS,R.J. Kinsella Magnet School of the Performing ...,10,17.0,7,10.0,NaN,NaN,NaN,NaN,NaN,NaN
264,HPS,R.J. Kinsella Magnet School of the Performing ...,11,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,HPS,R.J. Kinsella Magnet School of the Performing ...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
266,HPS,Sport and Medical Sciences Academy,6,141.0,78,63.0,NaN,NaN,NaN,NaN,NaN,NaN
267,HPS,Spo1i and Medical Sciences Academy,7,57.0,18,39.0,NaN,NaN,NaN,NaN,NaN,NaN
268,HPS,Spo1i and Medical Sciences Academy,8,79.0,28,51.0,NaN,NaN,NaN,NaN,NaN,NaN
269,HPS,Sport and Medical Sciences Academy,9,194.0,123,71.0,NaN,NaN,NaN,NaN,NaN,NaN
270,HPS,Sport and Medical Sciences Academy,10,43.0,23,20.0,NaN,NaN,NaN,NaN,NaN,NaN
271,HPS,Sport and Medical Sciences Academy,11,33.0,19,14.0,NaN,NaN,NaN,NaN,NaN,NaN
272,HPS,Sport and Medical Sciences Academy,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# Here are just the problematic school names
# And the schools.closest() match for each
problem_names = []
for n in no_offers["school"].unique(): 
    print "'" + n + "', '" + schools.closest(n).iloc[0]["School Name"] + "'"

'R.J. Kinsella Magnet School of the Performing A1is', 'Kinsella Magnet School of Performing Arts'
'R.J. Kinsella Magnet School of the Performing Arts', 'Kinsella Magnet School of Performing Arts'
'Sport and Medical Sciences Academy', 'Sports and Medical Sciences Academy'
'Spo1i and Medical Sciences Academy', 'Sports and Medical Sciences Academy'
'STEM Magnet at Annie Fisher', 'STEM Magnet at Fisher School'
'University High School of Science and Engineering', 'University High School of Science and Engineering'


In [110]:
offers[offers["school"].str.contains("Kinsella")]["school"].unique()

array([u'R.J. Kinsella Magnet School of the Performing A1ts',
       u'R.J. Kinsella Magnet School of the Performing Arts'],
      dtype=object)

In [111]:
apps[apps["school"].str.contains("Kinsella")]["school"].unique()

array([u'R.J. Kinsella Magnet School of the Performing A1is',
       u'R.J. Kinsella Magnet School of the Performing Arts'],
      dtype=object)

In [114]:
len(apps) + len(offers)

549